In [1]:
import os
import sys 

## replace with root project dir
sys.path.append("/mnt/workspace/__ing/llming/DTC/course")


from utils.utils.environment import initialize_env_variables

# Initialize environment variables
initialize_env_variables()

Initialized environment variables listed in: /mnt/workspace/__ing/llming/DTC/course/.env


# OpenAI

In [3]:
from openai import OpenAI

client = OpenAI()

In [5]:
## question
q = 'The course has already started, can I still enroll?'

## 1. No Context

In [6]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

print(response.choices[0].message.content)

Whether or not you can still enroll in a course that has already started depends on several factors, including the institution's policies, the specific course, and the instructor's discretion. Here are some steps you can take to find out:

1. **Check the Course Catalog/Website**: Look for any information regarding late enrollment policies. Some institutions provide specific deadlines and procedures for late registration.

2. **Contact the Registrar's Office**: They can provide details about whether late enrollment is possible and what steps you need to take.

3. **Email the Instructor**: Reaching out directly to the course instructor can be very effective. Explain your situation and express your interest in the course. Some instructors may allow late enrollment, especially if the course structure supports it.

4. **Review Prerequisites and Requirements**: Ensure that you meet any prerequisites and can catch up on missed material. Some courses may have essential early sessions that are 

## 2. 